<a href="https://colab.research.google.com/github/SriHarini2405/Naan-Mudhalvan-project/blob/main/emotion_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets scikit-learn emoji googletrans==4.0.0-rc1


In [ ]:
!pip install langdetect


In [ ]:
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, pipeline
from sklearn.metrics import classification_report
from langdetect import detect
from googletrans import Translator
import emoji
import numpy as np
import torch


In [ ]:
dataset = load_dataset("go_emotions", "simplified")
dataset = dataset.rename_column("labels", "label")


In [ ]:
!pip install transformers


In [ ]:
from transformers import DistilBertTokenizer


In [ ]:
# Install if not already done
!pip install transformers

# Import
from transformers import DistilBertTokenizer

# Load tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")


In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=28  # 27 emotions + neutral
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
emotion_classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True
)


Device set to use cuda:0
/usr/local/lib/python3.11/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [ ]:
translator = Translator()

def smart_preprocess_text(text):
    try:
        lang = detect(text)
        if lang != "en":
            translated = translator.translate(text, dest="en")
            text = translated.text
            print(f"🔵 Translated Text: {text}")
    except:
        print("⚠️ Language detection failed. Proceeding with original text.")

    text_with_emoji_words = emoji.demojize(text, delimiters=(" ", " "))



In [ ]:
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

# Load tokenizer and model (make sure you already fine-tuned and saved your model if needed)
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=28)
model.eval()

# Define GoEmotions label names (27 emotions + 1 neutral)
goemotions_labels = [
    'admiration', 'amusement', 'anger', 'annoyance', 'approval', 'caring', 'confusion', 'curiosity',
    'desire', 'disappointment', 'disapproval', 'disgust', 'embarrassment', 'excitement', 'fear',
    'gratitude', 'grief', 'joy', 'love', 'nervousness', 'neutral', 'optimism', 'pride', 'realization',
    'relief', 'remorse', 'sadness', 'surprise'
]

# Prediction function for multilabel
def predict_emotion(text):
    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    # Forward pass
    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probs = torch.sigmoid(logits)[0]  # Multi-label: use sigmoid

    # Get top 2 predicted emotions
    topk = torch.topk(probs, k=2)
    top_indices = topk.indices.tolist()
    top_emotions = [goemotions_labels[i] for i in top_indices]

    return top_emotions


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def predict_emotion(text):
    # Step 1: Translate if not English
    try:
        lang = detect(text)
        if lang != "en":
            translated = translator.translate(text, dest="en")
            text = translated.text
            print(f"🔵 Translated Text: {text}")
    except:
        print("⚠️ Language detection failed. Proceeding with original text.")

    # Step 2: Convert emojis to text (optional, helps BERT interpret)
    text = emoji.demojize(text, delimiters=(" ", " "))

    # Step 3: Tokenize and predict
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    probs = torch.sigmoid(logits)[0]

    # Step 4: Get top 2 emotion indices
    top2 = torch.topk(probs, k=2)
    top_indices = top2.indices.tolist()
    top_scores = top2.values.tolist()

    top_emotions = [(goemotions_labels[i], round(top_scores[j], 3)) for j, i in enumerate(top_indices)]
    return top_emotions


In [ ]:
texts = [
    "Je suis tellement excité et plein d'énergie",  # French
    "Estoy muy emocionado y feliz",                # Spanish
    "🤩🎉",                                        # Emoji only
    "I can't wait for the weekend! I'm so excited!" # English
]

for text in texts:
    emotions = predict_emotion(text)
    print(f"\n🔹 Input: {text}")
    print("🎯 Predicted Emotions:", emotions)


🔵 Translated Text: I am so excited and full of energy

🔹 Input: Je suis tellement excité et plein d'énergie
🎯 Predicted Emotions: [('excitement', 0.549), ('gratitude', 0.519)]
🔵 Translated Text: I am very excited and happy

🔹 Input: Estoy muy emocionado y feliz
🎯 Predicted Emotions: [('excitement', 0.548), ('gratitude', 0.524)]
⚠️ Language detection failed. Proceeding with original text.

🔹 Input: 🤩🎉
🎯 Predicted Emotions: [('excitement', 0.54), ('gratitude', 0.528)]

🔹 Input: I can't wait for the weekend! I'm so excited!
🎯 Predicted Emotions: [('excitement', 0.543), ('amusement', 0.522)]
